# load library 

In [13]:
from subprocess import call

import pandas as pd
import ast
import re
import spacy
import tempfile

## Create documentation (How to install, How to run, What is the structure of the dataset, license etc.)
## Might look into Pydriller

class Preprocessing:
#     def __init__(self):        
#         input_file = pd.read_csv(filename,encoding = "ISO-8859-1")
        
    def lemmatization(comments):
        nlp = spacy.load('en_core_web_md')
        lemma_comments = []
        doc = nlp(comments)
        for token in doc:
            lemma_word = token.lemma_
            if(lemma_word == "-PRON-"):
                lemma_word = "pron"
            lemma_comments.append(lemma_word.lower())
        comment_outs = ' '.join(lemma_comments)
        comment_outs = re.sub(r'[^A-Za-z0-9]+','   ', comment_outs, flags=re.IGNORECASE)
        comment_outs = re.sub(r'\s+',' ',comment_outs)
        return comment_outs
    
    def special_character_removal(comments):
        comments = re.sub(r'[^A-Za-z0-9.\']+',' ',comments)
        comments = re.sub(r'\s+',' ',comments)
        return comments
    
    def create_n_gram(comments,labels,n_gram_file_output):
        
        temp = tempfile.TemporaryFile(mode='w')
        temp2 = tempfile.TemporaryFile(mode='w')
#         file = open(DATA_SOURCE+'/temp_n_gram_preparation','w')
        for index in range(len(comments)):
            if labels[index] != 0:
                temp.write(chr(0x02)+chr(0x03)+"\n")
                temp.write(comments[index]+"\n")
        temp.close()
        
        call("../ngweight-master/waf",shell=True)
        call("../ngweight-master/bin/default/ngweight -w -s 0 < "+ temp.name + '>' + temp2.name ,shell=True)
        
        with open(temp2) as csvfile:                    
            reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
            for row in reader:
                if int(row[3]) <= 1:
                    continue
                n_gram_filter.append(row)

        with open(n_gram_file_output,'w') as csvfile:
            writer = csv.writer(csvfile,delimiter='\t', quotechar='|')
            writer.writerows(n_gram_filter)
                
        # delete file
        pathlib.Path.unlink(temp.name)
        pathlib.Path.unlink(temp2.name)
        
        
        

In [15]:
inp = pd.read_csv('../snapshot.csv',encoding = "ISO-8859-1")

preprocessing = Preprocessing()


FileNotFoundError: File b'../snapshot.csv' does not exist